# Mastermind

### Abstract
This is a study on algorithmic solutions for Mastermind, as part of the study on codeword completion games. All implementations are my own.

As for why Mastermind might be interesting as a subject, other than the fact that it is related other codeword completion games, it has similarities to practical problems such as when querying genomic data (Goodrich 2009). Note: The particular version presented by Goodrich is a variation called "Black-Peg" Mastermind which doesn't use white pins. These white-only or black-only variants are studied under different names, but they're different from the actual Mastermind game. For more information on such variations, see (Viglietta 2011). For good references on studies of the original game, see (Ville 2013).

This notebook was created to learn about existing methods for mastermind, before moving onto a solver for Wordle, which is just very slightly different. The problem space in Wordle is manageable, hence I'll be focusing on counting methods (basically brute force methods) that satisfy some kind of optimality condition, rather than divide-and-conquer methods or heuristics, which solve it less quickly, but are also necessary when brute force approaches become intractable.

### Problem definition

In Mastermind, the goal of one player (guesser) is to deduce a hidden codeword. A Mastermind codeword (in the original version) is a sequence of 4 pins, and there are 6 possible colors for each pin. After each guess, the other player (setter) tells the guesser how many of those pins are correct (matched both position and color), and included (matched only color). The guesser needs to uncover the correct codeword in as few tries as he can.

The response to a guess is encoded by a number of black and white pins. A black pin indicates an exact match, of both position and color. A white pin indicates correct color only, while no pin indicates neither, i.e. both color and position didn't match. Note that neither black/white pins nor hole indicates *which* symbol(s) in the guess are correct (or not). Hence all possible responses could be encoded as a 2-tuple `(n_match_exact, n_match_color)`. The correct guess in the original Mastermind would elicit the response (4,0).

There are numerous optimal strategies with different optimality criteria, as could be seen in (Dowell). Perhaps the most well-known is Knuth's solution in 1976 (Knuth 1976), where he showed that his algorithm solves any codeword in the original mastermind with an upper bound of 5 steps. However, Knuth did not explain how he arrived at his partitioning of the problem space into the tree used by his algorithm, nor how he had chosen his initial guess of 1122.

To set up the problem, we have a small set of symbols (the pins), which make up the codeword. The set of all possible codewords is called the vocabulary. Since the set of symbols is countable, we can encode all the symbols by enumeration. And since all codewords are of length 4, the set of all codewords is 4-way cross product that produces a vocabulary size of $6^4=1296$:

Let $\mathcal{S}=\{1,2,3,4,5,6\}$

Let $\mathcal{V}=\mathcal{S}\times\mathcal{S}\times\mathcal{S}\times\mathcal{S}$

At any time, the guesser's goal is to minimize the number of guesses to deduce the hidden codeword. This means to reduce the remaining possibilities - called the solution or code pool - to 1. However, at face value, it's difficult to tell if a greedy strategy, i.e. minimizing the solution pool in the current step, is necessarily a globally optimal strategy, even without strictly defining what "optimality" means. Usually greedy algorithms aren't globally optimal. In fact, playing greedy (single step lookahead) isn't globally optimal for Mastermind, as mentioned in (Bestavros 1986).

We'll follow the notation in (Bestavros 1986). Let $P_i$ denote the solution pool at the $i$th time step. From the initial solution pool $P_0=\mathcal{V}$, after the $i$th guess, the solution pool reduces to $P_{i}\subset P_{i-1}$.

Let $\{G^1_i, G^2_i, ..., G^k_i\}=P_{i-1}$ denote the set of possible guesses in the $i$th time step. If guess $G^k_i$ is made in the $i$th time step, the new solution pool $P_i$ consists of all members in $P_{i-1}$ which are still possible, based on the $i$th response from the setter. When $\#P_i=1$ for some $i$, the game is won.

Likewise, let $\{R^1_i, R^2_i, ..., R^n_i\}$ be the set of all possible responses from the setter in the $i$th time step. Let $R^j_i$ be the setter's response in the $i$th time step.

### Information and the message tuple

In (Bestavros 1986), the authors quantified information content in an interesting way, in section 1.3 of their paper. We'll follow their reasoning and work our way to implementing MaxMin, which is very similar in terms of implementation (basically the opposite) to Knuth's algorithm, which uses MiniMax. Intuitively, we want to maximize the information gained (by some measure), from every guess that we make. Disambiguation: In Bestavros et al's original paper, the glyph $R$ is also used to denote the upper bound for the number of symbols. It won't have that meaning here.

After the $i$th round of the game, the tuple $(P_{i-1}, G^k_i, R^j_i)$ may be viewed as a message $m_i$ from some information source $\Omega$. The information in the message $I(m_i)$ can be used to get a new pool $P_i$ from $P_{i-1}$. Intuitively, the greater $I(m_i)$ is, the smaller the size of the new pool should be.

Aside: In order for information content to mean anything at all, we need to know the probability of each message. Usually, that means constructing the set of all possible messages $m_i$, and that would be $P_{i-1}\times G^k_i \times R^j_i$, and then observing or assuming some prior probability distribution the messages are distributed over.

However, going through the elements of the 3-tuple, $P_{i-1}$ was already computed in the $i-1$th time step, so there's only one such possible pool in the current time step. There's also only one guess $G^k_i$ made in the current time step. This means the number of possible messages $m_i$ is just the number of possible responses $\{R^1_i, R^2_i, ..., R^j_i\}$ from the setter in the current time step.

By the nature of the game, a guess $G^k_i$ imposes a partition on the remaining pool of codewords $P_{i-1}$. These partitions correspond to all possible responses $R^j_i$ in reponse to that one guess.

Aside: Initially, I found it difficult to see why this is a set partition (mutually disjoint sets). It's easier to see from the bottom up. Starting from any pool of solutions, for any guess, the most important observation to make is that there is only one legal response for any pattern. Therein lies the set partition.

Define $c^k_i(R^j_i)=c(P_{i-1}, G^k_i, R^j_i)$ as a notation aid in writing down this set partition. The original paper abuses notation by declaring only the shorthand $c^k_i(R^j_i)$. This is because $P_{i-1}$ is fixed and we only need to track the index $k$ of the guess $G^k_i$.

$$
\begin{align}
P_{i-1} &= c(P_{i-1},  G^k_i, R^1_i) \ \bigcup c(P_{i-1}, G^k_i, R^2_i) \ \bigcup \ ... \ \bigcup c(P_{i-1}, G^k_i, R^j_i) \\
&= c^k_i(R^1_i) \ \bigcup c^k_i(R^2_i) \ \bigcup \ ... \ \bigcup c^k_i(R^j_i)
\end{align}
$$

and just so that it's clear that $c$ are partitions of $P_{i-1}$,

$$
c^k_i(R^a_i) \bigcap c^k_i(R^b_i) = \emptyset \ ; \ \forall \ 0 \lt a,b \leq j \ ; \ a \neq b
$$

Now we have enough material to ask the question: What is the probability of getting a particular message $m_i$? Well, since we have a 1-1 mapping between messages and responses, it's the probability of getting the response $R^j_i$. Since $c^k_i$ partitions $P_{i-1}$, by the law of total probability, that's simply:

$$
Prob[m_i]=\frac{Prob[c^k_i(R^j_i)]}{Prob[P_{i-1}]}
$$

How do we evaluate that? Well, we know that the set of patterns $P_i$ for any $i$ is countable and finite. If we assume a uniform distribution over each pattern in the initial pool $P_0$, we could compute the above expression by counting, if the total number of patterns is "small enough". By using this prior, we are assuming the setter is picking patterns completely at random. Thus the probability is:

$$
Prob[m_i]=\frac{\# c^k_i(R^j_i)}{\# P_{i-1}}
$$

And thus the information of a message is:

$$
I(m_i) = -\log Prob(m_i) = \log \# P_{i-1} - \log \# c^k_i(R^j_i)
$$

Finally, the total information in $P_0$ is $\log 6^4$. The game ends after $n$ rounds, when our total information equals this value:

$$
\sum^n_{i=1}\log I(m_i) = \log 6^4
$$

### Imports

In [1]:
# Imports
import itertools
import numpy as np
# import cupy as cp
import time

### Method: get_response()

Before we get to the algorithms, we need to be able to return the response for any guess, given the true codeword. Let's start by defining this method.

This function takes in a guess (an array), and the truth (an array), and it needs to output a 2-tuple. It needs to compute the exact number of matches, which is just elementwise equality. It also needs to compute color matches, which requires testing for set membership. The number of elements in this set is the number of symbols, which for Mastermind should be "small". Therefore [np.isin()](https://numpy.org/doc/stable/reference/generated/numpy.isin.html) for unsorted arrays should be fine.

In [2]:
#@param guess: Numpy array of the current guess(es).
#@param truth: Numpy array of the true codeword.
#@return: (n_exact_matches, n_color_matches)
def get_response_v1(guess: np.ndarray, truth: np.ndarray) -> np.array:
    
    # Get the boolean masks
    exact_matches = (guess == truth)
    color_matches = np.isin(guess, truth, assume_unique=False)
    
    n_exact_matches = np.sum(exact_matches, axis=-1)
    n_color_matches = np.sum(color_matches & ~exact_matches, axis=-1)
    r = np.array([n_exact_matches, n_color_matches]).T
    return r

In [3]:
# =Sanity check=
guess = np.array([1,3,4,3], dtype=int)
truth = np.array([1,2,2,3], dtype=int)
print(get_response_v1(guess, truth))

guess = np.array([
    [1,3,4,3],
    [1,2,3,2],
    [6,5,4,3]
])
print(get_response_v1(guess, truth))

[2 1]
[[2 1]
 [2 2]
 [1 0]]


Brute force methods require testing every possible guess against each other O(n^2). This means we only need one argument, which is the 2d array of remaining codewords. This gives us the actual method signature which we're going to use.

Internally, `np.isin()` is doing sorted search. Since the length of codewords is small, I don't think it's worth worrying about that particular function right now.

I think it's more important to think about our memory footprint. Recall that the size of the initial pool $\mathcal{V}=P_0 = (\#\mathcal{S})^L$ where $\mathcal{S}$ is the set of available symbols and $L$ is the codeword length. Since we're comparing each guess in the pool against each other, this means even for basic Mastermind, the number of comparisons we're performing is $6^{4\times2} \approx 1.68 \times 10^6$. From Wikipedia, the biggest version of Mastermind has 10 symbols and 6 holes, which amounts to $10^{12}$ comparisons.

In the code, `m` is $6^4$ and `n` is 4.

In [4]:
# For each codeword in the pool, use it as a guess to test against
# every codeword, including itself, and return the response tuples.
# @param pool: 2d numpy array of remaining candidate codewords.
# @return: 3d tensor of (n_exact_matches, n_color_matches)
def get_response(pool: np.ndarray) -> np.ndarray:
    m, n = np.shape(pool)
    exact_matches = (pool == pool[:,None,:])         # (m, m, n)
    
    color_matches = np.zeros((m, m, n), dtype=bool)
    for i in range(m):
        color_matches[i,:,:] = np.isin(pool, pool[i], assume_unique=False)
    color_matches = color_matches & ~exact_matches   # (m, m, n)
    
    n_exact_matches = np.sum(exact_matches, axis=2)  # (m, m)
    n_color_matches = np.sum(color_matches, axis=2)  # (m, m)
    r = np.array([n_exact_matches, n_color_matches]) # (2, m, m)
    r = np.transpose(r, (1,2,0))                     # (m, m, 2)
    return r

In [5]:
# =Sanity check=
pool = np.array([
    [1,1,1,1],
    [1,1,2,2],
    [1,2,3,3],
    [1,2,3,4],
    [1,3,5,7]
], dtype=int)

print(get_response(pool))

[[[4 0]
  [2 0]
  [1 0]
  [1 0]
  [1 0]]

 [[2 2]
  [4 0]
  [1 1]
  [1 1]
  [1 0]]

 [[1 3]
  [1 3]
  [4 0]
  [3 0]
  [1 1]]

 [[1 3]
  [1 3]
  [3 1]
  [4 0]
  [1 1]]

 [[1 3]
  [1 1]
  [1 2]
  [1 1]
  [4 0]]]


In [6]:
# Construct our initial codeword pool
def get_initial_pool(n_syms: int=6, word_length: int=4) -> np.ndarray:
    
    # DEFINE Symbols. 6 colors in mastermind.
    syms = set([x for x in range(n_syms)])

    # DEFINE Vocabulary. Word length 4 in mastermind.
    vocab = itertools.product(*itertools.repeat(syms, word_length))
    
    pool_size = n_syms ** word_length
    pool = np.zeros((pool_size, word_length), dtype=int)
    i = 0
    for v in vocab:
        pool[i] = v
        i += 1
    return pool

In [7]:
# Test run on actual initial pool
pool = get_initial_pool(n_syms=6, word_length=4)
start = time.time()
r = get_response(pool)
print(np.shape(r))
print("n seconds: {}".format(time.time() - start))

(1296, 1296, 2)
n seconds: 0.1011972427368164


For reasons already stated, if we try to do the same for the largest version of Mastermind, numpy will complain about us asking it to allocate about 5 terabibytes of memory. To be fair, for classic Mastermind, Knuth showed that his algorithm has a worst case of 5 steps. On the other hand, more memory-friendly approaches, such as Chvatal's divide and conquer algorithm, has a worst-case of 18 steps (Viglietta 2012).

Note that this is the memory cost of computing the solution. When we do have the actual solution, it is a fixed tree of guesses with max branching factor that equals the number of possible responses, and its depth should be proportional to the information content of the problem, i.e. logarithmic w.r.t the size of the code pool.

In [8]:
big_pool = get_initial_pool(n_syms=10, word_length=6)
big_r = get_response(big_pool)

C:\Users\LQKhoo\AppData\Local\Temp/ipykernel_9844/1615290612.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  exact_matches = (pool == pool[:,None,:])         # (m, m, n)


MemoryError: Unable to allocate 5.46 TiB for an array with shape (1000000, 1000000, 6) and data type bool

### Method: partition()

For any given guess, we need to partition the current pool by all possible responses.

First, we need to generate a list of all those responses. The size of this list depends solely on the length of the codeword, not the size of the symbol set. If the length of our codewords is $L$, this list contains all possible tuples of two non-negative integers, which sum up the all the non-negative integers between 0 and $L$.

In [9]:
# Generate all possible tuples of responses for the given problem.
def get_response_keys(word_length: int) -> np.ndarray:
    n = word_length + 1
    n_keys = int(n/2 * (2*1 + (n-1)*1)) # arithmetic sum
    keys = np.zeros((n_keys, 2), dtype=int)
    k = 0
    for i in range(n):
        for j in range(n-i):
            keys[k] = i, j
            k += 1
    return keys[::-1]

In [10]:
# =Check=
print(get_response_keys(4))

[[4 0]
 [3 1]
 [3 0]
 [2 2]
 [2 1]
 [2 0]
 [1 3]
 [1 2]
 [1 1]
 [1 0]
 [0 4]
 [0 3]
 [0 2]
 [0 1]
 [0 0]]


The first method maps a given codeword to the row or column index of the current pool. Likewise, the second method maps a reponse tuple (in the form of an ndarray) to the index in the vector of all possible responses.

In [11]:
# Given a codeword and a pool, return the index of the codeword in the pool.
def get_codeword_idx(codeword: np.ndarray, pool: np.ndarray) -> int:
    mask = (codeword[None, :] == pool)
    idx = np.argwhere(np.all(mask, axis=1)).flatten()
    assert(np.size(idx) == 1)
    return idx[0]

# Given a response, return its index in the array of all possible responses.
def get_response_idx(response: np.ndarray, keys: np.ndarray) -> int:   
    mask = (response[None, :] == keys)
    idx = np.argwhere(np.all(mask, axis=1)).flatten()
    assert(np.size(idx) == 1)
    return idx[0]

In [12]:
# =Test=
S, L = 6, 4
pool = get_initial_pool(n_syms=S, word_length=L)
guess = np.array([1,2,3,4])
codeword_idx = get_codeword_idx(guess, pool)
print(codeword_idx)
print(pool[codeword_idx])

res = np.array([2,1])
keys = get_response_keys(word_length=L)
response_idx = get_response_idx(res, keys)
print(response_idx)
print(keys[response_idx])

310
[1 2 3 4]
4
[2 1]


Recall that the information of a message $m_i$ is:

$$
I(m_i) = \log \frac{\# P_{i-1}}{\# c^k_i(R^j_i)}
$$

The following function returns two matrices:
1. The responses matrix is an (m x m) matrix that contains the correct response key for every guess vs every truth.
2. The vector V is supposed to contain "information" in the paper. However, here, V is a (m, n_keys) matrix. Each row of this matrix V corresponds to a guess. And there are a total of n_keys entries in each row. These entries are the new sizes of the pools corresponding to each possible response, which is exactly $c^k_i(R^j_i)$.

So the way to use V is as follows: Our guess gives us the correct row. The response given by the setter gives us the correct column. The entry in V corresponding to these coordinates is the size of the new pool. In order to get $I$, all we need to do find the size of the previous pool, which is just the sum of the row, and divide that number by the entry in V. Then take the log and we have $I$.

In [13]:
def compute_information(pool: np.ndarray, keys: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    m, n = np.shape(pool)
    r = get_response(pool)
    
    n_keys = np.shape(keys)[0]
    key_idx = np.arange(n_keys)
    
    responses = np.zeros((m, m), dtype=int)          # Response idx for every guess vs every possible truth
    ## n_possible_responses = np.zeros((m,), dtype=int) # Num of possible responses for every guess
    
    V = np.zeros((m, n_keys), dtype=int)
    for i in range(m):
        mask = (r[None,i,:] == keys[:,None,:]).T          # (2, n_keys, m) Match the numbers in each tuple individually.
        mask = np.all(mask, axis=0)                       # (n_keys, m)    We want both numbers to match.
        response_indices = np.sum(mask * key_idx, axis=1) # (m, )          Convert it to indices
        responses[i,:] = response_indices
        uniques, counts = np.unique(response_indices, return_counts=True)

        V[i,uniques] = counts # Keys double as indices here as they are 0-indexed.
        ## n_possible_responses[i] = np.shape(np.unique(response_indices))[0]
    
    return V, responses

In [14]:
# =Test=

S, L = 6, 4
keys = get_response_keys(word_length=L)
pool = get_initial_pool(n_syms=S, word_length=L)
V, responses = compute_information(pool, keys)

print(np.shape(V))
for i in range(np.shape(V)[0]):
    print(V[i])

(1296, 15)
[  1   0  20   0   0 150   0   0   0 500   0   0   0   0 625]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  8

[  1  12   8  54  72  24 108 216 144  32  81 216 216  96  16]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1  12   8  54  72  24 108 216 144  32  81 216 216  96  16]
[  1  12   8  54  72  24 108 216 144  32  81 216 216  96  16]
[  1  12   8  54  72  24 108 216 144  32  81 216 216  96  16]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   4  16   6  48  96   4  48 192 256   1  16  96 256 256]
[  1   8  12  24  72  54  32 144 216 108  16  96 216 216  81]
[  1   8

In [15]:
# Given a guess, return an array of response idxs for every codeword in the pool, if they were the truth.
# This array is used downstream to slice the pool into partitions for performing further guesses.
def partition(guess: np.ndarray, pool: np.ndarray, keys: np.ndarray, responses: np.ndarray) -> np.ndarray:
    guess_idx = get_codeword_idx(guess, pool)
    responses = responses[guess_idx] # Grab the row relevant to the guess we're making
    return responses

In [16]:
# =Test=

S, L = 6, 4
guess = np.array([3,5,2,4])
keys = get_response_keys(word_length=L)
pool = get_initial_pool(n_syms=S, word_length=L)
V, responses = compute_information(pool, keys)

r = partition(guess, pool, keys, responses)
print(np.shape(r))
print(r)

(1296,)
[14 14 13 ...  6  3  6]


### MaxMin (information maximin)

The MaxMin algorithm makes the guess which maximizes the minimum amount of information gained. This is done by computing the minimum information for every possible guess $G^k_i$. As mentioned in (Bestavros 1986), this is a pessimistic algorithm.

Now, in (Bestavros 1986), they mentioned their tie-breaking strategy, but even so, it's fairly obvious that the problem has symmetries, e.g. using their terminology which encodes symbols with letters, intuitively we should get the same results whether we start off the problem by first guessing `AAAB`, `BBBC`, `CCCD`, or even `EEEA` or `ABAA` etc. Therefore we still need a tiebreaker for when we've checked every single dimension of the vector $V$ and we still have more than one candidate guess. These groupings of guesses arise from said symmetry. Here, we simply pick the first such guess as tiebreaker.

The following is my implementation of the greedy MaxMin algorithm (single-step lookahead) as described in Bestavros 1986. For the closely related MaxEnt algorithm, instead of computing the minimum information, the average is used instead. I won't be implementing MaxEnt because it's very similar (as in the code) to MaxMin.

In [17]:
def make_guess_maxmin_greedy(pool: np.ndarray, keys: np.ndarray):
    
    # The information in each time step is different, i.e. the counts are different
    # because the starting pool is different, so we have to recompute V every time step,
    # rather than just re-partitioning the previous V.
    V, responses = compute_information(pool, keys)   
        
    V_sorted = np.sort(V, axis=1) # V but rowsorted
    V_sorted = np.flip(V_sorted) # Sort descending by #c
    
    m, n = np.shape(pool)
    n_keys = np.shape(keys)[0]
    guess_idx = np.arange(m)
    
    # Loop is for tiebreaking as described in Bestavros 1986.
    for i in range(n_keys):
        col = V_sorted[:,i]
        maximin_idx = np.argwhere(col == np.amax(col)).flatten()
        intersect_idx = np.intersect1d(guess_idx, maximin_idx)
        if np.size(intersect_idx) == 0:
            break
        guess_idx = intersect_idx
    best_guess_idx = guess_idx[0]
    
    # Generate partitions
    responses = responses[best_guess_idx] # Grab the row relevant to the guess we're making
    return responses, best_guess_idx

def mastermind_solve(algo: str, truth: np.ndarray,
                         n_syms: int=6, word_length: int=4, max_iters=20) -> tuple[np.ndarray, int, list, list]:
    # Check that the problem is defined properly.
    assert(np.shape(truth)[0] == word_length) # Word length needs to match given truth value.
    assert(np.all(truth >= 0) and np.all(truth < n_syms)) # Check that symbols are encoded correctly.
    
    # Algorithms
    MAXMIN = 'maxmin'
    MAXENT = 'maxent'
    MINIMAX = 'minimax'
    if algo == MAXMIN:
        solver = make_guess_maxmin_greedy
    elif algo == MAXENT:
        solver = make_guess_maxent_greedy
    elif algo == MINIMAX:
        solver = make_guess_minimax_greedy
    else:
        raise NotImplementedError(algo)
    
    pool = get_initial_pool(n_syms, word_length)
    keys = get_response_keys(word_length)
    
    # Used for logging the solution path only.
    guesses = []
    messages = []
    
    nsteps = 1
    for i in range(max_iters):
        responses, best_guess_idx = solver(pool, keys) # Make guess
        truth_idx = get_codeword_idx(truth, pool)
        response = responses[truth_idx] # Give the appropriate response to the chosen guess.
        guesses.append(pool[best_guess_idx])
        messages.append(keys[response])
        if np.shape(pool)[0] <= 1:
            break
        partition_indices = np.argwhere(responses == response).flatten()
        pool = pool[partition_indices]
        nsteps += 1
    
    return pool.flatten(), nsteps, guesses, messages

In [18]:
# =Test run=
S, L = 6, 4
truth = np.array([5,3,2,1])
answer, nsteps, guesses, messages = mastermind_solve(algo='maxmin', truth=truth, n_syms=S, word_length=L)
for i in range(len(guesses)):
    print("Guess {}: {}, response: {}".format(i+1, guesses[i], messages[i]))
print("Answer: {} in {} steps.".format(answer, nsteps))

truth = np.array([5,5,5,5])
answer, nsteps, guesses, messages = mastermind_solve(algo='maxmin', truth=truth, n_syms=S, word_length=L)
for i in range(len(guesses)):
    print("Guess {}: {}, response: {}".format(i+1, guesses[i], messages[i]))
print("Answer: {} in {} steps.".format(answer, nsteps))

truth = np.array([5,5,4,0])
answer, nsteps, guesses, messages = mastermind_solve(algo='maxmin', truth=truth, n_syms=S, word_length=L)
for i in range(len(guesses)):
    print("Guess {}: {}, response: {}".format(i+1, guesses[i], messages[i]))
print("Answer: {} in {} steps.".format(answer, nsteps))

Guess 1: [0 0 0 0], response: [0 0]
Guess 2: [1 1 1 1], response: [1 0]
Guess 3: [1 2 2 2], response: [1 1]
Guess 4: [3 1 2 3], response: [1 2]
Guess 5: [3 2 1 4], response: [0 3]
Guess 6: [5 1 3 2], response: [1 3]
Guess 7: [5 3 2 1], response: [4 0]
Answer: [5 3 2 1] in 7 steps.
Guess 1: [0 0 0 0], response: [0 0]
Guess 2: [1 1 1 1], response: [0 0]
Guess 3: [2 2 2 2], response: [0 0]
Guess 4: [3 3 3 3], response: [0 0]
Guess 5: [4 4 4 4], response: [0 0]
Guess 6: [5 5 5 5], response: [4 0]
Answer: [5 5 5 5] in 6 steps.
Guess 1: [0 0 0 0], response: [1 0]
Guess 2: [0 1 1 1], response: [0 1]
Guess 3: [2 0 2 2], response: [0 1]
Guess 4: [3 4 0 5], response: [0 4]
Guess 5: [4 3 3 0], response: [1 1]
Guess 6: [5 5 4 0], response: [4 0]
Answer: [5 5 4 0] in 6 steps.


In [258]:
# =Find worst / average cases=
S, L = 6, 4
pool = get_initial_pool(n_syms=S, word_length=L)
n = np.shape(pool)[0]

mean, N = 0, 0 # For computing running mean
nsteps_max = 0

start = time.time()
for i in range(n):
    if i % 100 == 0:
        print(f"Running problem {i} / {n}")
    truth = pool[i]
    _, nsteps, _, _ = mastermind_solve(algo='maxmin', truth=truth, n_syms=S, word_length=L)
    
    # Knuth's running mean
    N += 1
    dx = nsteps - mean
    mean += dx / N
    nsteps_max = max(nsteps, nsteps_max)
print("Running time: {} seconds.".format(time.time() - start))
print("Worst-case: {} steps, average-case: {} steps.".format(nsteps_max, mean))

Running problem 0 / 1296
Running problem 100 / 1296
Running problem 200 / 1296
Running problem 300 / 1296
Running problem 400 / 1296
Running problem 500 / 1296
Running problem 600 / 1296
Running problem 700 / 1296
Running problem 800 / 1296
Running problem 900 / 1296
Running problem 1000 / 1296
Running problem 1100 / 1296
Running problem 1200 / 1296
Running time: 462.4941716194153 seconds.
Worst-case: 10 steps, average-case: 6.2708333333333375 steps.


Take the timings with a grain of salt because I'm running plenty of other tasks in the background so there's definitely some thrashing. Considering it's brute-forcing a problem of this size, I suppose it's not too bad.

The number of responses is $O(N^2)$ with respect to the number of possible guesses, at every time step. Also, recall that for MaxMin, the algorithm needs to sort the vector $V$, which requires an average of $O(N\log N)$ w.r.t input size, _and then_ the tiebreaking process involves `np.intersect1d`, which involes a good number of `np.sort` and `np.unique` calls.

I think the main issue is that the matrices are too large to fit in CPU cache, which multiprocessing isn't going to help. Instead of doing premature optimization on a pessimistic algorithm that's clearly far from optimal, I want to move on to the others, because those are more interesting.

### MiniMax (Knuth's)

In Knuth's algorithm, also known as MiniMax, we take the guess whose largest partition is smallest, i.e. take the max of each row in V, and make the guess whose max is smallest. As tiebreaker, compare the second-largest values in the respective rows in V, and so on.

However, as explained in (Knuth 1976), the first guess needs to be set to `1122`, or in our case, since our symbol set starts from zero, `0011`, in order for the algorithm to solve classic mastermind in 5 moves. This first move doesn't actually conform to the MiniMax rule. Otherwise, as Knuth stated, there is a series of patterns which requires 6 moves. Thing is, this hack only works for classic mastermind (6 colors, 4 holes). Since the rest of our implementation is agnostic to these two parameters, I'll not be fixing the first guess to `0011`. Our pure MiniMax implementation will take the first guess `0012`, which matches what Knuth said in page 3 of his paper, and it will not have the worst-case of 5 moves for classic mastermind.

In [21]:
def make_guess_minimax_greedy(pool: np.ndarray, keys: np.ndarray):
    
    V, responses = compute_information(pool, keys)
    
    # Knuth's algorithm requires calling np.unique, which creates jagged arrays.
    # We'll plop these jagged arrays into a matrix M, which we have pre-filled
    # with entries equal to maxint.
    M = np.full(np.shape(V), np.iinfo(int).max)
    for i in range(np.shape(V)[0]):
        entries = np.unique(V[i])[::-1] # np.unique already returns sorted ascending.
        length = np.size(entries)
        M[i][0:length] = entries

    m, n = np.shape(pool)
    n_keys = np.shape(keys)[0]
    guess_idx = np.arange(m)
    
    for i in range(n_keys):
        col = M[:,i]
        maximin_idx = np.argwhere(col == np.amin(col)).flatten()
        intersect_idx = np.intersect1d(guess_idx, maximin_idx)
        if np.size(intersect_idx) == 0:
            break
        guess_idx = intersect_idx
    best_guess_idx = guess_idx[0]
    
    # Generate partitions
    responses = responses[best_guess_idx]
    return responses, best_guess_idx

In [22]:
# =Test run=
S, L = 6, 4
truth = np.array([5,3,2,1])
answer, nsteps, guesses, messages = mastermind_solve(algo='minimax', truth=truth, n_syms=S, word_length=L)
for i in range(len(guesses)):
    print("Guess {}: {}, response: {}".format(i+1, guesses[i], messages[i]))
print("Answer: {} in {} steps.".format(answer, nsteps))

Guess 1: [0 0 1 2], response: [0 2]
Guess 2: [1 3 4 0], response: [1 1]
Guess 3: [1 1 5 5], response: [0 2]
Guess 4: [2 5 4 1], response: [1 2]
Guess 5: [5 3 2 1], response: [4 0]
Answer: [5 3 2 1] in 5 steps.


In [260]:
# =Find worst / average cases=
S, L = 6, 4
pool = get_initial_pool(n_syms=S, word_length=L)
n = np.shape(pool)[0]

mean, N = 0, 0 # For computing running mean
nsteps_max = 0

start = time.time()
for i in range(n):
    if i % 100 == 0:
        print(f"Running problem {i} / {n}")
    truth = pool[i]
    _, nsteps, _, _ = mastermind_solve(algo='minimax', truth=truth, n_syms=S, word_length=L)
    
    # Knuth's running mean
    N += 1
    dx = nsteps - mean
    mean += dx / N
    nsteps_max = max(nsteps, nsteps_max)
print("Running time: {} seconds.".format(time.time() - start))
print("Worst-case: {} steps, average-case: {} steps.".format(nsteps_max, mean))

Running problem 0 / 1296
Running problem 100 / 1296
Running problem 200 / 1296
Running problem 300 / 1296
Running problem 400 / 1296
Running problem 500 / 1296
Running problem 600 / 1296
Running problem 700 / 1296
Running problem 800 / 1296
Running problem 900 / 1296
Running problem 1000 / 1296
Running problem 1100 / 1296
Running problem 1200 / 1296
Running time: 396.47707772254944 seconds.
Worst-case: 6 steps, average-case: 4.605709876543209 steps.


As explained in (Ville 2013), in section 3.1, the author mentioned that Knuth's original paper (Knuth 1976) pointed out that guessing only using codes remaining in the current pool of possible codewords is not optimal. Therefore in an optimal solution, the number of codewords to attempt is not a monotonically decreasing set. Neither MaxMin / MaxEnt, or MiniMax do this, so none of them are optimal in this sense.

To give some intuition, in certain cases, making a guess which we know is incorrect, will give us more information that we could have gotten otherwise, in order to inform us on the guesses which we have yet to make.

## References

* Bestavros, A. and Belal, A., 1986. Mastermind a game of diagnosis strategies. In Alexandria University.
    * https://www.cs.bu.edu/fac/best/res/papers/alybull86.pdf
* Chvátal, V., 1983. Mastermind. Combinatorica, 3(3), pp.325-329.
    * (scihub)
* Dowell, J. Defeating Mastermind.
    * http://mercury.webster.edu/aleshunas/Support%20Materials/Analysis/Dowelll%20-%20Mastermind%20v2-0.pdf
* Goodrich, M.T., 2009. On the algorithmic complexity of the Mastermind game with black-peg results. Information Processing Letters, 109(13), pp.675-678.
    * https://www.ics.uci.edu/~goodrich/pubs/ipl.pdf
* Knuth, D.E., 1976. The computer as master mind. Journal of Recreational Mathematics, 9(1), pp.1-6.
    * http://www.cs.uni.edu/~wallingf/teaching/cs3530/resources/knuth-mastermind.pdf
* Koyama, Kenji; Lai, Tony (1993). "An Optimal Mastermind Strategy". Journal of Recreational Mathematics (25): 230–256.
    * http://www.cs.uni.edu/~wallingf/teaching/cs3530/resources/knuth-mastermind.pdf
* Viglietta, G., 2012, June. Hardness of mastermind. In International Conference on Fun with Algorithms (pp. 368-378). Springer, Berlin, Heidelberg.
    * https://arxiv.org/pdf/1111.6922.pdf
    * http://m.giovanniviglietta.com/slides/mastermind.pdf
* Ville, G., 2013. An optimal mastermind (4, 7) strategy and more results in the expected case. arXiv preprint arXiv:1305.1010.
    * https://arxiv.org/pdf/1305.1010.pdf